In [11]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



In [12]:
a = ana.Anatomist()

### Variable definitions

In [13]:
embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/fronto-parietal_medial_face_left/10-37-44_89/imagen_random_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/participants.csv"

In [14]:
label = "C0_L"

In [15]:
side = "L" # "R" or "L"
region = "fronto-parietal_medial_face." # "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='imagen'

# Building predictors

In [16]:
participants = pd.read_csv(participants_file, dtype={'Subject':object})
participants = participants.set_index("Subject")
participants = participants[[label]]

In [17]:
participants.head()

,C0_L
Subject,
000000001274,N
000000022453,N
000000075717,N
000000106601,N
000000106871,N


In [18]:
participants = participants[[label]].dropna()
participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants[label].unique()

array([0, 2, 1])

In [19]:
len(participants)

2008

In [20]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")

In [21]:
embeddings

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
000000001274,-8.232107,-0.017696,6.677566,-53.525750,56.741985,44.150036,12.050084,-91.314960,47.097645,0.284472,...,46.661545,-26.816505,5.624397,29.494692,15.234098,-10.840830,-19.157925,-17.527260,6.201155,-28.953253
000000022453,-26.518356,20.054657,4.024464,-9.291761,23.494900,-15.812221,-11.193628,34.573044,6.324287,14.830438,...,-26.284756,10.347884,-7.534056,-33.282640,-11.051048,10.271432,4.641976,87.443960,11.558672,28.541819
000000075717,-22.144148,-56.482970,-31.308126,-30.209505,-37.061405,-40.030290,17.532510,10.887849,-19.200954,11.256378,...,7.817508,10.078654,24.457428,-48.016780,-12.500457,47.596825,-65.165280,39.573494,13.526820,9.031254
000000106601,-23.945068,-43.529755,12.192984,-24.822536,24.889790,13.883513,6.094525,36.845230,0.639967,19.254684,...,-37.249577,-19.260061,17.176546,3.116904,0.862859,39.679390,-23.097473,34.508950,-28.819044,8.667316
000000106871,-6.392687,-27.164656,-52.168888,-19.316263,73.841255,-30.990849,0.884190,-10.887053,-4.967268,7.585090,...,40.393550,-0.717537,-29.926804,-6.148581,3.575815,10.368586,-67.152110,35.778194,-58.701940,26.881361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
000099873252,33.659256,-22.731638,-11.313125,-26.220524,18.697487,23.252691,0.956394,-21.084023,64.557470,0.081966,...,50.633804,47.226387,-44.738370,53.081512,-6.569685,48.419273,-27.086376,-9.253797,-37.648506,4.339344
000099875982,46.830296,-17.288609,0.176214,-23.802479,47.256210,-1.865489,59.328680,28.271162,47.449820,28.934890,...,9.316505,57.569138,-17.140518,-34.337960,-42.239850,68.900580,-10.445233,10.360641,-16.534828,23.025390
000099888850,-25.079500,-6.202746,-56.379770,5.875311,7.577985,-43.801540,-14.633049,12.735417,-51.131393,-37.381214,...,8.217363,-32.692030,21.040258,-5.323525,22.228708,-32.885050,-13.521323,52.748650,36.792717,-5.561362


In [22]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

2.075722633483528

In [23]:
df.mean()

C0_L         0.47006
predicted    0.47006
dtype: float64

In [24]:
df.head()

,C0_L,predicted
000000001274,0.0,0.532646
000000022453,0.0,0.198311
000000075717,0.0,0.260510
000000106601,0.0,0.286693
000000106871,0.0,0.220098


### Function definitions

In [25]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [26]:
def test_visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        print(i, n_pack)

In [27]:
test_visualize_averages_along_parameter(df, "predicted", database)

0 11
4 11
10 11


In [28]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [29]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

<soma.aims.Volume_DOUBLE object at 0x7361eaf8dab0> (27, 93, 61, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo

In [30]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [31]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x7361b739e200>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x7361b739d1b0>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7361a5ff8a60>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x7361eaf8dab0>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x7361a5fc5cf0>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x73619d510310>,
 'a_sum_vol10': <anatomist.cpp.anatomist.SliceableObject object at 0x7361a5fc5fc0>,
 'rvol10': <anatomist.cpp.anatomist.MObject object at 0x73619d5103a0>,
 'wvr10': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x73619d51c160>}

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
